In [9]:
import optuna
import pandas as pd
import xgboost as xgb
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import logging
import sys
import numpy as np
from pandas.core.frame import DataFrame

import pandas as pd
test = pd.read_pickle('test_add_cate.pkl')
train = pd.read_pickle('train_add_cate.pkl')
label = train['price']
train = train.drop(['price'],axis=1)
# train = train.drop(['listing_id','make_model_combined','svd9_category','suggested_price','price'],axis=1)
# test = test.drop(['listing_id','make_model_combined','suggested_price','svd9_category'],axis=1)

X_train,X_test,y_train,y_test = train_test_split(train,label,test_size=0.0002,random_state=111)
drop_list = ['vintage cars',
 'coe car',
 'consignment car',
 'sta evaluated car',
 'sgcarmart warranty cars',
 'low mileage car',
 'electric cars',
 'direct owner sale',
 'parf car',
 'opc car',
 'premium ad car',
 'hybrid cars',
 'imported used vehicle',
 'almost new car',
 'rare & exotic']
X_train,y_train = train,label
X_train.drop(drop_list,axis=1)
data_train = xgb.DMatrix(X_train.fillna(-1), label=y_train)
data_test = xgb.DMatrix(X_test.fillna(-1), label=y_test)
data_rmse_test = xgb.DMatrix(X_test.fillna(-1))
test_DMatrix = xgb.DMatrix(test.fillna(-1))
watch_list = [(data_test, 'eval'), (data_train, 'train')]


In [10]:
param = {
            'max_depth': 11, 
            'eta': 0.01, 
            'silent': 1, 
            'gamma':0,
            'objective': 'reg:linear',
            'reg_alpha':1,
            'reg_lambda':1,
            'n_estimators':300,
#                 'num_boost_round':trial.suggest_int("num_boost_round", 800, 3000),
            'subsample':0.9,
            'colsample_bytree':0.6,
            'min_child_weight':3
        }
print(param)
bst = xgb.train(param, data_train, num_boost_round=1500,evals=watch_list)
rmse = np.sqrt(mean_squared_error(y_test.tolist(),list(bst.predict(data_rmse_test))))
res = {

            'Predicted':list(bst.predict(test_DMatrix))
        }
res=DataFrame(res)
res.to_csv('model_saving3/xgboost_res_'+'final'+'.csv')

{'max_depth': 11, 'eta': 0.01, 'silent': 1, 'gamma': 0, 'objective': 'reg:linear', 'reg_alpha': 1, 'reg_lambda': 1, 'n_estimators': 300, 'subsample': 0.9, 'colsample_bytree': 0.6, 'min_child_weight': 3}
[02:16:55] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[02:16:55] WARNING: ../src/learner.cc:576: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	eval-rmse:99767.09375	train-rmse:173784.85938
[1]	eval-rmse:98744.17969	train-rmse:172182.60938
[2]	eval-rmse:97750.98438	train-rmse:170591.42188
[3]	eval-rmse:96739.70312	train-rmse:169052.07812
[4]	eval-rmse:95730.85156	train-rmse:167466.76562
[5]	eval-rmse:94616.89844	train-rmse:165917.62500
[6]	e

KeyboardInterrupt: 

In [12]:
import lightgbm as lgb
hyper_params = {'colsample_bytree': 0.7365299211906035,
 'learning_rate': 0.06433836700594038,
 'max_depth': 18,
 'min_child_samples': 30,
 'min_child_weight': 0.007499472789765832,
 'n_estimators': 400,
 'num_leaves': 40,
 'reg_alpha': 0.3975937739726987,
 'reg_lambda': 0.13116705819059657,
 'subsample': 0.9,
 'subsample_for_bin': 50000}
gbm = lgb.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train)

LGBMRegressor(colsample_bytree=0.7365299211906035,
              learning_rate=0.06433836700594038, max_depth=18,
              min_child_samples=30, min_child_weight=0.007499472789765832,
              n_estimators=400, num_leaves=40, reg_alpha=0.3975937739726987,
              reg_lambda=0.13116705819059657, subsample=0.9,
              subsample_for_bin=50000)